In [3]:
import os
os.chdir('..')

In [4]:
pwd

'/mnt/md1/user_victor/automatic_melody_harmonization'

In [5]:
from tonal import pianoroll2number, joint_prob2pianoroll96
from tonal import tonal_centroid, chord482note, chord962note, note2number
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pickle
from model.VAE_all_chords import VAE
from decode import *
import matplotlib.pyplot as plt

In [6]:
chord_idx = np.load('./data/chord_indices.npy')
chord_onehot = np.load('./data/chord_onehot.npy')
length = np.load('./data/length.npy')

In [9]:
device = 'cpu'
val_size = 500
print('splitting testing set...')

val_chord_idx = torch.from_numpy(chord_idx[:val_size]).long().to(device)
val_chord = torch.from_numpy(chord_onehot[:val_size]).float().to(device)
val_length = torch.from_numpy(length[:val_size]).to(device)


splitting testing set...


In [11]:
# Load model
print('building model...')
with torch.no_grad():
    model = VAE(device = device).to(device)
    model.load_state_dict(torch.load('output_models/model_vae_all_chords.pth'))
    print(model)
    model.eval()


building model...
VAE(
  (embedding): Embedding(633, 96)
  (encoder): LSTM(96, 256, num_layers=2, batch_first=True, bidirectional=True)
  (encoder_output2mean): Linear(in_features=512, out_features=16, bias=True)
  (encoder_output2logv): Linear(in_features=512, out_features=16, bias=True)
  (latent2decoder_input): Linear(in_features=16, out_features=128, bias=True)
  (decoder): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (outputs2chord): Linear(in_features=512, out_features=633, bias=True)
)


In [19]:
# Sampconductorne song
# torch.manual_seed(0)
i = 8
sample_length = val_length[i].unsqueeze(0)
sample_chord_idx = val_chord_idx[i].squeeze().unsqueeze(0)
sample_chord = val_chord[i].unsqueeze(0)

print(sample_length)
print('sample_chord_idx',sample_chord_idx[0][:sample_length])
print(sample_chord.shape)

chord_pred, logp, mu, log_var, input_chord = model(sample_chord_idx,sample_length)
print('chord pred shape',chord_pred[0].shape)

logits = torch.max(chord_pred[0][:sample_length[0]],-1)
gen_chord_index = logits.indices
print('max logits',logits)
# print('max index',gen_chord_index)



tensor([45])
sample_chord_idx tensor([  0, 265, 415, 415, 415, 415, 570, 570, 570, 570,  70,  70,  70,  70,
        206, 206, 557, 589, 269, 269, 265, 265, 415, 415, 415, 415, 570, 570,
        570, 570,  70,  70, 206, 206, 563, 563, 269, 269,   0,   0, 415, 415,
        426, 426,   0])
torch.Size([1, 272, 633])
chord pred shape torch.Size([272, 633])
max logits torch.return_types.max(
values=tensor([0.9623, 0.9761, 0.9398, 0.9511, 0.9336, 0.7912, 0.5354, 0.5785, 0.6622,
        0.6003, 0.9303, 0.9523, 0.7829, 0.9105, 0.7642, 0.9088, 0.1526, 0.1783,
        0.5303, 0.4768, 0.6532, 0.7783, 0.8454, 0.7763, 0.8294, 0.8797, 0.6270,
        0.8008, 0.6651, 0.6342, 0.9811, 0.9685, 0.9811, 0.9853, 0.7480, 0.7247,
        0.5092, 0.1117, 0.8941, 0.7608, 0.6392, 0.6237, 0.1450, 0.2257, 0.9236],
       grad_fn=<MaxBackward0>),
indices=tensor([  0,   0, 415, 415, 415, 415,  22,  22,  22,  22,  70,  70,  70,  70,
        206, 206, 420, 566, 269, 269, 265, 265, 415, 415, 415, 415,  22,  22,
       